## Profiling Python

Python (and `IPython`) provide built-in tools for profiling, enhanced for interactive computing by `IPython`', with a number of third-party tools like `pyinstrument` that refine and extend these tools. `ipyprofile` provides [`Pyinstrument`](#Pyinstrument), building on top of these tools.

In [ ]:
import sys

if __name__ == "__main__" and "pyodide" in sys.modules:
    %pip install -r requirements.txt

In [ ]:
def fib(n):
    return n if n < 2 else fib(n - 1) + fib(n - 2)

In [ ]:
async def afib(n):
    return n if n < 2 else await afib(n - 1) + await afib(n - 2)

In [ ]:
%%prun
print(fib(10))

While strictly _accurate_, counting every call, the built-in profiler results can be hard to interpret. 

This magic also cannot handle "top-level `await`", increasingly seen even in interactive computing. 

In [ ]:
import asyncio

loop = asyncio.get_running_loop()

> Uncommenting and running the following cell will fail

In [ ]:
# %%prun
# await afib(10)

## `pyinstrument`

[`pyinstrument`](https://github.com/joerick/pyinstrument) is a _sampling_ profiler for Python, meaning it tries to capture a representative report, rather than an authoritative one. This can help tell the story of a larger trace, at the risk of hiding some details.

It provides its own magic, providing an embedded HTML report.

In [ ]:
%reload_ext pyinstrument

In [ ]:
%%pyinstrument?

In [ ]:
%%pyinstrument
print(fib(20))

`%%pyinstrument`, if configured, can handle the top-level `await` syntax.

> _This example works in a regular `ipykernel` session, but not in `pyodide-kernel`, as it relies on threads_

In [ ]:
# %%pyinstrument --async_mode=enabled
# print(await afib(20))

While this provides additional insights, `pyinstrument` _also_ provides support for `speedscope` JSON, allowing it to to be used directly with `Flamegraph`, as well as its own JSON format.

## `Pyinstrument`

`Pyinstrument` is an opinionated wrapper around `pyinstrument`, optimized for interactive profiling.

## Basic Example

For simplicity, the `Pyinstrument` examples below use the naive Fibonacci sequence (`fib`) and its asynchronous counterpart (`afib`), defined above.

The `Pyinstrument` widget provides no default UI: it offers both a `.flame_graph` and `.call_graph`, as well as the `.tabs()` method which displays both.

In [ ]:
from ipyprofiler.widget_pyinstrument import Pyinstrument

ps = Pyinstrument()
ps.tabs(layout={"min_height": "60vh"})

Like `pyinstrument`, it offers a context manager, which helps ensure profilers are properly cleaned up.

In [ ]:
with ps.profile("fib"):
    fib(15)

In [ ]:
with ps.callgraph.hold_sync():
    ps.callgraph.use_elk = ps.callgraph.show_time = ps.callgraph.group_by_file = 1
    ps.callgraph.direction = "left_to_right"

Using `.profile` again will update the existing output.

In [ ]:
with ps.profile("fib, but 20"):
    fib(20)

### Additional Options

`Pypseedscope` and `.profile` expose a few additional options:

In [ ]:
ps.profile?

### Opinions

A number of strings are rewritten to make results more tractable:
- the current working directory is replaced with `./`
- the long, auto-generated file/profile names are replaced

## Asynchronous Code

As non-magic code is already correctly mangled by IPython, no additional flags are needed to profile `async` code.

In [ ]:
aps = Pyinstrument(name="asynchronous fibonacci")
with aps.profile():
    await afib(20)
aps.tabs()

## Multiple Cells

Profiling a single block of code with the `.profile` context manager will usually show the most useful output, but setting the `.profiling` member directly is possible. This will show the mechanics of `IPython` and `ipykernel` along with the code of interest.

In [ ]:
mps = Pyinstrument(
    name="multiple cells",
    processor_options={"hide_regex": r".*"},
)
mps.profiling = True

In [ ]:
fib(20)

In [ ]:
await afib(20)

In [ ]:
mps.profiling = False
mps.tabs()

In [ ]:
mps.callgraph.show_options()

In [ ]:
with mps.callgraph.hold_sync():
    mps.callgraph.use_elk = mps.callgraph.group_by_file = 1
    mps.callgraph.show_time = 0
    mps.callgraph.direction = "top_to_bottom"

> ## More Demos
>
> [⬅️ Flamegraph](01_flamegraph.ipynb) | [⬆️ back to index](00_index.ipynb)